# Sentiment Analysis Using Various Approaches

## Lexicon-based approach 
- Unsupervised learning
- Based on calculating sentiment scores of words in a document from predetermined dictionaries, called lexicons.
- Each word's sentiment is determined, and the scores are combined to calculate the overall sentiment of the sentence. 
- A lexicon is a dictionary that contains a collection of words that is categorized as positive, negative, and neutral by experts. Their scores can change over time. 

In [17]:
import numpy as np 
import pandas as pd
import json
import time

# NLTK Bing Liu Lexicon 
import nltk
# nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize 

# VADER 
import nltk.corpus
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\MJ\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


## Loading a subset of reviews and meta data

In [2]:
n = 1 
total_rows = 0

def process_chunks(file, chunksize = 10000):

    # Setting as global variables
    global n, total_rows  
    
    chunks = pd.read_json(file, lines=True, chunksize = chunksize)
    dfs = []  
    n_chunks = 0

    for chunk in chunks:
        dfs.append(chunk)
        n_chunks += 1  # Count the number of chunks processed
        print(len(chunk), " rows added")
        n += 1 
        total_rows += len(chunk)
        if n_chunks >= 10:  # Process only the first 5 chunks
            break  
            
    print("Done")
    print("Total rows:", total_rows)
    return pd.concat(dfs, ignore_index=True)

In [3]:
reviews = "../data/Home_and_Kitchen.jsonl"
meta = "../data/meta_Home_and_Kitchen.jsonl"

start = time.process_time()

reviews_subset = process_chunks(reviews)

end = time.process_time()
elapsed_time = end - start
print('Created a subset of the reviews dataset')
print('Execution time:', elapsed_time, 'seconds')

print('--------------')
start = time.process_time()

meta_subset = process_chunks(meta)

end = time.process_time()
elapsed_time = end - start
print('Created a subset of the meta dataset')
print('Execution time:', elapsed_time, 'seconds')

10000  rows added
10000  rows added
10000  rows added
10000  rows added
10000  rows added
10000  rows added
10000  rows added
10000  rows added
10000  rows added
10000  rows added
Done
Total rows: 100000
Created a subset of the reviews dataset
Execution time: 1.84375 seconds
--------------
10000  rows added
10000  rows added
10000  rows added
10000  rows added
10000  rows added
10000  rows added
10000  rows added
10000  rows added
10000  rows added
10000  rows added
Done
Total rows: 200000
Created a subset of the meta dataset
Execution time: 8.3125 seconds


In [4]:
reviews_subset.head(3)

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,1,Received Used & scratched item! Purchased new!,Livid. Once again received an obviously used ...,[],B007WQ9YNO,B09XWYG6X1,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2023-02-26 01:03:29.298,1,True
1,5,Excellent for moving & storage & floods!,I purchased these for multiple reasons. The ma...,[],B09H2VJW6K,B0BXDLF8TW,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-12-26 08:30:10.846,0,True
2,2,Lid very loose- needs a gasket imo. Small base.,[[VIDEOID:c87e962bc893a948856b0f1b285ce6cc]] I...,[{'small_image_url': 'https://m.media-amazon.c...,B07RL297VR,B09G2PW8ZG,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-05-25 02:54:56.788,0,True


In [5]:
reviews_subset.columns

Index(['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase'],
      dtype='object')

## Bing Liu Lexicon - NLTK

The Bing Liu lexicon has a total of 6, 786 words with 2,005 classified as positive and 4,781 as negative. CLassification is binary (positive or negative).

In [6]:
print('Total number of words in opinion lexicon', len(opinion_lexicon.words()))
print('Examples of positive words:', opinion_lexicon.positive()[:10])
print('Examples of negative words:', opinion_lexicon.negative()[:10])

Total number of words in opinion lexicon 6789
Examples of positive words: ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']
Examples of negative words: ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']


In [7]:
pos_score = 1
neg_score = -1
word_dict = {}

# Adding the positive words to the dictionary
for word in opinion_lexicon.positive():
    word_dict[word] = pos_score 

# Adding the negative words to the dictionary 
for word in opinion_lexicon.negative():
    word_dict[word] = neg_score 

def bing_liu_score(text):
    sentiment_score = 0 
    bag_of_words = word_tokenize(text.lower())

    # Check if bag_of_words is empty
    if bag_of_words: 
        for word in bag_of_words: 
            if word in word_dict: 
                sentiment_score += word_dict[word]
        return sentiment_score / len(bag_of_words)
    else: 
        return 0

In [8]:
sentiment_scores = reviews_subset.copy()
sentiment_scores['Bing_Liu_Score'] = sentiment_scores['text'].apply(bing_liu_score)
sentiment_scores[['rating', 'text', 'asin', 'Bing_Liu_Score']].sample(5)

,rating,text,asin,Bing_Liu_Score
88819,4,Not much different than other celebratory cand...,B001L7XA6M,0.081633
52653,5,[[VIDEOID:cc3f0f3e321b523f577431a4b0a8da46]] D...,B088GG43CJ,0.087500
65678,5,doesn't leak! Holds temperature exactly like i...,B00KR9OQE0,0.078947
95552,5,UPDATE<br />I've been using this vacuum almost...,B07C5MXJMX,0.054054
74545,5,Perfect “in between” size hangers,B07FK4BGXZ,0.142857


In [11]:
# Calculate mean sentiment score for each rating category
accuracy_scores = sentiment_scores.groupby('rating').agg({'Bing_Liu_Score':'mean'})

print(accuracy_scores)

        Bing_Liu_Score
rating                
1            -0.020034
2             0.004955
3             0.026585
4             0.069269
5             0.116748


## VADER Lexicon
Rule-based lexicon. 
9,000 features with scales of [-4] Extremely Negative to [4] Extremely Positive with [0] for Neutral or Neither. 